<a href="https://colab.research.google.com/github/priyanu17/transfer_learning_for_currency_identification/blob/master/CNN_Currency_Identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [4]:
import os
os.mkdir('Currency_Images')

In [5]:
import zipfile
with zipfile.ZipFile("/content/drive/My Drive/150253_347412_bundle_archive.zip", 'r') as zip_ref:
    zip_ref.extractall("Currency_Images")

In [6]:
# Transfer Learning : MobileNetV2

In [7]:
# required imports

from tensorflow.keras.layers import  Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2

In [8]:
# loading MobileNetV2 model
mobile_net_v2 = MobileNetV2(input_shape=[224, 224] + [3], weights='imagenet', include_top=False)

# acquiring trained parameters
for layer in mobile_net_v2.layers:
    layer.trainable = False
  

9412608/9406464 [==============================] - 0s 0us/step


In [9]:
# adding a flatten layer
model_mobile_net_v2 = Flatten()(mobile_net_v2.output)

#adding the output layer
prediction = Dense(211, activation='softmax')(model_mobile_net_v2)

# create a model object
model_mobile_net_v2 = Model(inputs=mobile_net_v2.input, outputs=prediction)

model_mobile_net_v2.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________________

In [10]:
# compile the model

model_mobile_net_v2.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [11]:
# required imports

from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [12]:
# using custom variations on training images as given in docs

data_gen_mobilenet_v2 = ImageDataGenerator(preprocessing_function=preprocess_input)

In [13]:
# extract training images from directory

training_set = data_gen_mobilenet_v2.flow_from_directory(
                directory = "Currency_Images/coins/data/train",
                class_mode = 'categorical',
                target_size = (224, 224),
                batch_size = 32,
                subset = 'training'
                )

Found 6413 images belonging to 211 classes.


In [14]:
# validation set images extracted from directory

validation_set = data_gen_mobilenet_v2.flow_from_directory(
        directory = 'Currency_Images/coins/data/validation',
        class_mode='categorical',
        target_size = (224, 224)
)

Found 844 images belonging to 211 classes.


In [15]:
# test images extracted from directory

test_set = data_gen_mobilenet_v2.flow_from_directory(
        directory = 'Currency_Images/coins/data/test',
        class_mode='categorical',
        target_size = (224, 224),
        batch_size=32
)

Found 844 images belonging to 211 classes.


In [16]:
# model fit on the training set

history_mobilenet_v2 = model_mobile_net_v2.fit_generator(
                                        training_set,
                                        validation_data = validation_set,
                                        epochs = 10
                             )

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
201/201 [==============================] - 35s 173ms/step - loss: 12.6918 - accuracy: 0.5235 - val_loss: 9.5737 - val_accuracy: 0.6564
Epoch 2/10
201/201 [==============================] - 33s 165ms/step - loss: 2.6152 - accuracy: 0.8848 - val_loss: 10.2705 - val_accuracy: 0.6991
Epoch 3/10
201/201 [==============================] - 33s 165ms/step - loss: 1.8528 - accuracy: 0.9312 - val_loss: 11.0339 - val_accuracy: 0.7287
Epoch 4/10
201/201 [==============================] - 33s 164ms/step - loss: 2.0208 - accuracy: 0.9368 - val_loss: 11.3722 - val_accuracy: 0.7109
Epoch 5/10
201/201 [==============================] - 33s 164ms/step - loss: 2.0448 - accuracy: 0.9439 - val_loss: 12.2611 - val_accuracy: 0.7097
Epoch 6/10
201/201 [==============================] - 33s 163ms/step - loss: 2.1527 - accuracy: 0.9393 - val_loss: 12.8722 - val_accuracy: 0.7393
Epoch 7/10
201/201 [============================

In [17]:
# final evaluation on the test set

score_mobilenet_v2 = model_mobile_net_v2.evaluate_generator(test_set, verbose=2)

Instructions for updating:
Please use Model.evaluate, which supports generators.
27/27 - 4s - loss: 16.6218 - accuracy: 0.7299


In [18]:
# test loss and test accuracy

print('MobileNetV2 Model : Test Loss is : ', score_mobilenet_v2[0])
print('MobileNetV2 Model : Test Accuracy is : ', score_mobilenet_v2[1])

MobileNetV2 Model : Test Loss is :  16.621789932250977
MobileNetV2 Model : Test Accuracy is :  0.7298578023910522


In [19]:
# Transfer Learning : ResNet50

In [20]:
# required imports

from tensorflow.keras.layers import  Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50

In [21]:
# loading resnet50 architecture
res_net_50 = ResNet50(input_shape=[224, 224] + [3], weights='imagenet', include_top=False)

# acquiring trained parameters for the model
for layer in res_net_50.layers:
    layer.trainable = False
  

94773248/94765736 [==============================] - 1s 0us/step


In [22]:
# adding a flatten layer
modified_res_net_50 = Flatten()(res_net_50.output)

# adding an output layer
prediction = Dense(211, activation='softmax')(modified_res_net_50)

# create a model object
modified_res_net_50 = Model(inputs=res_net_50.input, outputs=prediction)

# view the structure of the model
modified_res_net_50.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [23]:
# compile the model

modified_res_net_50.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [24]:
# required imports

from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator



In [25]:
# using custom variations as per the resnet50 model
data_gen_resnet50 = ImageDataGenerator(preprocessing_function=preprocess_input)

In [26]:
# extract training images from directory

training_set = data_gen_resnet50.flow_from_directory(
                directory = "Currency_Images/coins/data/train",
                class_mode = 'categorical',
                target_size = (224, 224),
                batch_size = 32,
                subset = 'training'
                )

Found 6413 images belonging to 211 classes.


In [27]:
# validation set images extracted from directory

validation_set = data_gen_resnet50.flow_from_directory(
        directory = 'Currency_Images/coins/data/validation',
        class_mode='categorical',
        target_size = (224, 224)
)

Found 844 images belonging to 211 classes.


In [28]:
# test images extracted from directory

test_set = data_gen_resnet50.flow_from_directory(
        directory = 'Currency_Images/coins/data/test',
        class_mode='categorical',
        target_size = (224, 224),
        batch_size=32
)

Found 844 images belonging to 211 classes.


In [29]:
# model fit on the training set

history_resnet50 = modified_res_net_50.fit_generator(
                                        training_set,
                                        validation_data = validation_set,
                                        epochs = 10
                             )

Epoch 1/10
201/201 [==============================] - 41s 206ms/step - loss: 22.9162 - accuracy: 0.5224 - val_loss: 18.1925 - val_accuracy: 0.6422
Epoch 2/10
201/201 [==============================] - 40s 199ms/step - loss: 4.5358 - accuracy: 0.8790 - val_loss: 17.8919 - val_accuracy: 0.6991
Epoch 3/10
201/201 [==============================] - 40s 199ms/step - loss: 3.3512 - accuracy: 0.9214 - val_loss: 18.1169 - val_accuracy: 0.7287
Epoch 4/10
201/201 [==============================] - 40s 200ms/step - loss: 3.7881 - accuracy: 0.9219 - val_loss: 22.3848 - val_accuracy: 0.7085
Epoch 5/10
201/201 [==============================] - 40s 200ms/step - loss: 3.9436 - accuracy: 0.9303 - val_loss: 25.1626 - val_accuracy: 0.7239
Epoch 6/10
201/201 [==============================] - 40s 198ms/step - loss: 3.5022 - accuracy: 0.9376 - val_loss: 23.8348 - val_accuracy: 0.7322
Epoch 7/10
201/201 [==============================] - 39s 195ms/step - loss: 3.6889 - accuracy: 0.9543 - val_loss: 27.3389 

In [30]:
# final evaluation on the test set

score_resnet50 = modified_res_net_50.evaluate_generator(test_set, verbose=2)


27/27 - 4s - loss: 30.9463 - accuracy: 0.7263


In [31]:
# test loss and test accuracy

print('Resnet50 Model : Test Loss is : ', score_resnet50[0])
print('Resnet50 Model : Test Accuracy is : ', score_resnet50[1])

Resnet50 Model : Test Loss is :  30.94625473022461
Resnet50 Model : Test Accuracy is :  0.7263033390045166


In [32]:
# Using Transfer Learning Models: VGG16

In [33]:
# required imports

from tensorflow.keras.layers import  Dense, Flatten

from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16

In [34]:
# loading VGG16 model
vgg = VGG16(input_shape=[224, 224] + [3], weights='imagenet', include_top=False)

# acquiring trained parameters
for layer in vgg.layers:
    layer.trainable = False
  

58892288/58889256 [==============================] - 0s 0us/step


In [35]:
# adding a flatten layer
modified_vgg = Flatten()(vgg.output)

# adding final output layer
vgg_pred = Dense(211, activation='softmax')(modified_vgg)

# create a model object
model_vgg = Model(inputs=vgg.input, outputs=vgg_pred)

model_vgg.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [36]:
# compile the model

model_vgg.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [37]:
# required imports

from tensorflow.python.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [38]:
# using custom variations on training images as given in docs

data_gen_vgg = ImageDataGenerator(preprocessing_function=preprocess_input)

In [39]:
# extract training images from directory

training_set = data_gen_vgg.flow_from_directory(
                directory = "Currency_Images/coins/data/train",
                target_size = (224, 224),
                class_mode = 'categorical',
                batch_size = 32,
                subset = 'training'
                )

Found 6413 images belonging to 211 classes.


In [40]:
# validation set images extracted from directory

validation_set = data_gen_vgg.flow_from_directory(
        directory = 'Currency_Images/coins/data/validation',
        target_size=(224, 224),
        class_mode='categorical',
        batch_size=32
)

Found 844 images belonging to 211 classes.


In [41]:
# test images extracted from directory

test_set = data_gen_vgg.flow_from_directory(
        directory = 'Currency_Images/coins/data/test',
        target_size=(224, 224),
        class_mode='categorical',
        batch_size=32
)

Found 844 images belonging to 211 classes.


In [42]:
# model fit on the training set

history_vgg = model_vgg.fit_generator(generator = training_set,
                              validation_data = validation_set,
                              epochs = 10
                             )

Epoch 1/10
201/201 [==============================] - 42s 211ms/step - loss: 35.0731 - accuracy: 0.5242 - val_loss: 28.5188 - val_accuracy: 0.6694
Epoch 2/10
201/201 [==============================] - 40s 201ms/step - loss: 9.0299 - accuracy: 0.8583 - val_loss: 25.9213 - val_accuracy: 0.7299
Epoch 3/10
201/201 [==============================] - 41s 202ms/step - loss: 6.6453 - accuracy: 0.9125 - val_loss: 28.9314 - val_accuracy: 0.7441
Epoch 4/10
201/201 [==============================] - 40s 200ms/step - loss: 6.6423 - accuracy: 0.9166 - val_loss: 35.2939 - val_accuracy: 0.7251
Epoch 5/10
201/201 [==============================] - 41s 202ms/step - loss: 6.0239 - accuracy: 0.9350 - val_loss: 34.9053 - val_accuracy: 0.7334
Epoch 6/10
201/201 [==============================] - 41s 202ms/step - loss: 6.3927 - accuracy: 0.9362 - val_loss: 40.3401 - val_accuracy: 0.7393
Epoch 7/10
201/201 [==============================] - 41s 202ms/step - loss: 5.8097 - accuracy: 0.9478 - val_loss: 43.4934 

In [43]:
# final evaluation on the test set

score_vgg16 = model_vgg.evaluate_generator(test_set, verbose=2)


27/27 - 4s - loss: 43.6898 - accuracy: 0.7571


In [44]:
# test loss and test accuracy

print('VGG16 Model : Test Loss is : ', score_vgg16[0])
print('VGG16 Model : Test Accuracy is : ', score_vgg16[1])

VGG16 Model : Test Loss is :  43.68977355957031
VGG16 Model : Test Accuracy is :  0.7571089863777161


In [46]:
# Custom Model:

In [70]:
# module used to generate custom variations on the images

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [71]:
# variations on the training set images

train_data_gen = ImageDataGenerator(
    rescale=1./255.0, 
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
    )


In [72]:
# extract training images from directory

training_set = train_data_gen.flow_from_directory(
                directory = "Currency_Images/coins/data/train",
                target_size = (64, 64),
                class_mode = 'categorical',
                batch_size = 32,
                subset = 'training'
                )

Found 6413 images belonging to 211 classes.


In [73]:
# variations on validation set images : rescaling

valid_data_gen = ImageDataGenerator(
    rescale=1./255.0
    )


In [74]:
# validation set images extracted from directory

validation_set = valid_data_gen.flow_from_directory(
        directory = 'Currency_Images/coins/data/validation',
        target_size=(64, 64),
        class_mode='categorical',
        batch_size=32
)

Found 844 images belonging to 211 classes.


In [75]:
# variations on the test images : rescaling

test_data_gen = ImageDataGenerator(rescale=1./255.0)

In [76]:
# test images extracted from directory

test_set = test_data_gen.flow_from_directory(
        directory = 'Currency_Images/coins/data/test',
        target_size=(64, 64),
        class_mode='categorical',
        batch_size=32
)

Found 844 images belonging to 211 classes.


In [77]:
# imports for the custom model

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D,MaxPool2D,Dropout,Flatten,Dense

In [78]:
# initiating the model
custom_model = Sequential()

# first set of convolution and pooling operations along with dropout
custom_model.add(Conv2D(16,(3,3),activation='relu',input_shape=(64,64,3)))
custom_model.add(MaxPool2D(2,2))
custom_model.add(Dropout(0.2))

# second set of convolution and pooling operations along with dropout
custom_model.add(Conv2D(32,(3,3),activation='relu'))
custom_model.add(MaxPool2D(2,2))
custom_model.add(Dropout(0.3))

# third set of convolution and pooling operations along with dropout
custom_model.add(Conv2D(64,(3,3),activation='relu'))
custom_model.add(MaxPool2D(2,2))
custom_model.add(Dropout(0.3))

# flattening the output from previous layers
custom_model.add(Flatten())

# first dense layer followed by a 0.2 dropout layer
custom_model.add(Dense(1024,activation='relu'))
custom_model.add(Dropout(0.4))

# second dense layer followed by a 0.2 dropout layer
custom_model.add(Dense(1024,activation='relu'))
custom_model.add(Dropout(0.4))

# third dense layer followed by a 0.2 dropout layer
custom_model.add(Dense(1024,activation='relu'))
custom_model.add(Dropout(0.4))

# fourth dense layer followed by a 0.2 dropout layer
custom_model.add(Dense(512,activation='relu'))
custom_model.add(Dropout(0.2))

# output layer
custom_model.add(Dense(211,activation='softmax'))

In [79]:
# compile the model

custom_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [80]:
custom_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 62, 62, 16)        448       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 31, 31, 16)        0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 31, 31, 16)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 29, 29, 32)        4640      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 12, 12, 64)       

In [84]:
# model fit on the training set

history_custom_model = custom_model.fit_generator(generator = training_set,
                            validation_data = validation_set,
                            epochs =20
                            )

Epoch 1/20
201/201 [==============================] - 30s 152ms/step - loss: 4.2374 - accuracy: 0.0850 - val_loss: 4.3628 - val_accuracy: 0.0912
Epoch 2/20
201/201 [==============================] - 30s 151ms/step - loss: 4.2002 - accuracy: 0.0867 - val_loss: 4.4118 - val_accuracy: 0.0711
Epoch 3/20
201/201 [==============================] - 30s 151ms/step - loss: 4.1502 - accuracy: 0.0903 - val_loss: 4.3183 - val_accuracy: 0.0853
Epoch 4/20
201/201 [==============================] - 30s 151ms/step - loss: 4.1244 - accuracy: 0.0950 - val_loss: 4.3316 - val_accuracy: 0.1102
Epoch 5/20
201/201 [==============================] - 30s 151ms/step - loss: 4.0931 - accuracy: 0.0973 - val_loss: 4.3151 - val_accuracy: 0.0877
Epoch 6/20
201/201 [==============================] - 31s 152ms/step - loss: 4.0509 - accuracy: 0.1042 - val_loss: 4.2248 - val_accuracy: 0.0983
Epoch 7/20
201/201 [==============================] - 31s 152ms/step - loss: 3.9967 - accuracy: 0.1049 - val_loss: 4.1307 - val_ac

In [85]:
# final evaluation on the test set

score_custom_model = custom_model.evaluate_generator(test_set, verbose=2)


27/27 - 3s - loss: 3.8711 - accuracy: 0.1493


In [86]:
# test loss and test accuracy

print('Custom Model : Test Loss is : ', score_custom_model[0])
print('Custom Model : Test Accuracy is : ', score_custom_model[1])

Custom Model : Test Loss is :  3.871145009994507
Custom Model : Test Accuracy is :  0.1492891013622284


In [87]:
# Transfer Learning models generate better accuracy than custom made developed models
# Transfer Learning models implement state of the art algorithms